# DeepLabCut Toolbox - Colab for standard (single animal) projects!

This notebook is a modified version of the demonstration notebook provided by <font color="green">DeepLabCut</font>. For the original version, refer to the [GitHub page of DeepLabCut](https://github.com/DeepLabCut/DeepLabCut).

![DeepLabCut GIF](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1619609897110-TKSTWKEM6HTGXID9D489/triMouseDLC.gif?format=2500w)

This notebook illustrates how to use the cloud to:
- create a training set  
- train a network  
- evaluate a network  
- create simple quality check plots  
- analyze novel videos

> **Note:** This notebook assumes you already have a DeepLabCut project folder with labeled data, which we have previously saved on Google Drive.

This notebook demonstrates the necessary steps to use <font color="green">DeepLabCut</font> for your own project.

We use the most basic code here, but many functions have additional features. For detailed explanations of each step, please refer to the official documentation:

📚 **DeepLabCut Documentation**: [https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html](https://deeplabcut.github.io/DeepLabCut/docs/standardDeepLabCut_UserGuide.html)

📝 Paper reference:  
Nath\*, Mathis\* et al. (2019): *Using DeepLabCut for markerless pose estimation during behavior across species.*  
**Nature Protocols**  
🔗 [Read the paper](https://www.nature.com/articles/s41596-019-0176-0)

------------

## 🔧 Step 1: Install Required Packages and Dependencies

We will install the specific versions of TensorFlow, PyTorch, and DeepLabCut optimized for CUDA 11.8 and cuDNN 8, which are compatible with the Colab environment.

### What the commands do:

- **Install TensorFlow 2.12.1**, built for CUDA 11.8 and cuDNN 8, along with required packages `tensorpack` and `tf_slim`.  
- **Downgrade PyTorch to version 2.3.1**, which supports CUDA 11.8 and installs necessary CUDA libraries for both PyTorch and TensorFlow.  
- **Install DeepLabCut directly from the latest GitHub source** to ensure you get the newest features and fixes.  
- **Create symbolic links for NVIDIA shared libraries** as recommended by TensorFlow's official installation instructions to fix potential runtime library issues.

Now, run the cell below:


In [ ]:
# Install the desired TensorFlow version, built for CUDA 11.8 and cudnn 8
!pip install "tensorflow==2.12.1" "tensorpack>=0.11" "tf_slim>=1.1.0"

# Downgrade PyTorch to a version using CUDA 11.8 and cudnn 8
#   This should also install the required CUDA libraries, for both PyTorch and TensorFlow
!pip install torch==2.3.1 torchvision --index-url https://download.pytorch.org/whl/cu118

# Install DeepLabCut
!pip install "git+https://github.com/DeepLabCut/DeepLabCut.git"

# As described in https://www.tensorflow.org/install/pip#step-by-step_instructions, 
# create symbolic links to NVIDIA shared libraries:
!ln -svf /usr/local/lib/python3.11/dist-packages/nvidia/*/lib/*.so* /usr/local/lib/python3.11/dist-packages/tensorflow


After installing the packages, it's important to confirm that both TensorFlow and PyTorch can access the GPU.  
This check will print the installed versions and verify that the GPU is available and working correctly.

### What this code does:

- Imports TensorFlow and PyTorch libraries.
- Prints the PyTorch version and checks if CUDA (GPU support) is available.
- Creates a small tensor on the GPU using PyTorch to confirm it can allocate GPU memory.
- Prints the TensorFlow version and lists available physical GPU devices.


In [ ]:
import tensorflow as tf
import torch

# Print PyTorch version and GPU availability status
print(f"Torch {torch.__version__}    ", torch.cuda.is_available())

# Create a tensor on the GPU to test PyTorch GPU functionality
print(f"{torch.zeros(5, device='cuda:0')}")

# Print TensorFlow version and list detected GPU devices
print(f"TensorFlow {tf.__version__}    ", tf.config.list_physical_devices("GPU"))

# Expected output:
# Torch 2.3.1+cu118     True
# tensor([0., 0., 0., 0., 0.], device='cuda:0')
# TensorFlow 2.12.1     [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

✅ After installation, you may be prompted to restart the runtime.  
If so, go to the menu: **Runtime → Restart runtime**, then re-run the notebook from here.

Once everything is installed, we’re ready to set up the project and start working with videos!

---


## 🚀 Step 2: Access Your Data from Google Drive

We will use Google Drive to store and access your project files (videos, labels, config, etc.) in Colab.

### Why mount Google Drive?  
Colab sessions are temporary and files are erased after the runtime disconnects. Mounting Google Drive lets you easily access your persistent files stored in the cloud.

### What does the code do?  
- **Mounts your Google Drive** so Colab can read files stored there.  
- **Copies a project folder** from your Drive to the Colab local environment for faster access during processing.

> 🔑 **Important:** Update the `source_folder` path to point to your actual folder in Google Drive! Most likely it will not be needed if you did everything according to instructions but it is worth to double-check!

In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Define source and destination paths
source_folder = '/content/drive/MyDrive/behavior&calcium-imaging-workshop'  # Update with your shared folder path
destination_folder = '/content/behavior&calcium-imaging-workshop'  # Content folder in Colab

# Copy files from source to destination folder
shutil.copytree(source_folder, destination_folder)

print("Files copied successfully!")

---

## Step 3: Now we can start configuring the project!

Now we define some key variables that tell DeepLabCut where your project files are and what kind of videos you want to analyze.

### What are these variables?

- `ProjectFolderName`:  
  The name of your DeepLabCut project folder.  
  This should match the folder where your labeled data and config file are stored.

- `VideoType`:  
  The video file format you want to analyze (e.g., `'mp4'`, `'avi'`, `'mov'`).

- `videofile_path` (do not edit):  
  This is the full path to the folder or list of videos that you want DeepLabCut to process.  
  Here, it points to the local folder in Colab where we copied your videos from Google Drive.


In [ ]:
# Customize these:
ProjectFolderName = 'workshop-network-2025'  # Your project folder name
VideoType = 'mp4'                            # Format of videos you want to analyze

# Do not edit this:
videofile_path = ['/content/behavior&calcium-imaging-workshop/data/behavior-videos']  # Path to your video files

# Show the path to confirm
videofile_path

In [ ]:
# Import DeepLabCut package
import deeplabcut

# Print the currently installed DeepLabCut version
print("DeepLabCut version:", deeplabcut.__version__)

In [ ]:
#This creates a path variable that links to your google drive copy
#No need to edit this, as you set it up before:
path_config_file = '/content/behavior&calcium-imaging-workshop/'+ProjectFolderName+'/config.yaml'
path_config_file

---

## Step 4: Create a training dataset:
### You must do this step inside of Colab:
After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
# Then, run this cell. There are many more functions you can set here, including which network to use, you can experiment with the parameters.
# Refer to the DeepLabCut User Guide for explaination.

deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

In the next cell, we'll be updating the pose configuration file, which contains all the settings necessary for training. This step ensures that we initialize the training process using weights obtained from previous training sessions.

This step is crucial because training the network to achieve optimal performance can be time-consuming, often taking several hours. Unfortunately, this duration far exceeds the time constraints of the workshop.

### What is a shuffle?

In DeepLabCut, a **shuffle** refers to one random split of your labeled data into training and testing subsets.  
Because training results can vary depending on which frames are used for training vs. testing, DeepLabCut allows you to create multiple shuffled splits to:

- Improve robustness by training multiple models on different data splits  
- Evaluate performance consistency across these splits  
- Select the best-performing model after training

### Important:

The **shuffle** number you use here should be the one you created in the previous step, which is most likely `1` if you followed the default settings.

By specifying the shuffle number, you tell DeepLabCut which data split to use for this particular training run.

In [ ]:
import yaml

# Define the shuffle number
shuffle = 1

# Path to the DeepLabCut pose configuration YAML file for the model
pose_cfg_path = f'/content/behavior&calcium-imaging-workshop/{ProjectFolderName}/dlc-models/iteration-5/ss_vs_sscwJan30-trainset95shuffle{shuffle}/train/pose_cfg.yaml'

# Path to the checkpoint file containing pretrained model weights
checkpoint = f'/content/behavior&calcium-imaging-workshop/{ProjectFolderName}/old_dlc-models/iteration-5/ss_vs_sscwJan30-trainset95shuffle2/train/snapshot-100000'

# Load the pose_cfg.yaml file content into a Python dictionary
with open(pose_cfg_path, 'r') as file:
    pose_cfg_content = yaml.safe_load(file)

# Update the 'init_weights' field in the config to point to the checkpoint file
pose_cfg_content['init_weights'] = checkpoint

# Save the updated configuration back to the YAML file
with open(pose_cfg_path, 'w') as file:
    yaml.dump(pose_cfg_content, file)

---

## Step 5: Start training

This function trains the network for a specific **shuffle** of the training dataset.

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU...
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/DeepLabCut/DeepLabCut/wiki/DOCSTRINGS#train_network

deeplabcut.train_network(path_config_file, shuffle=shuffle, displayiters=10,saveiters=50, maxiters=100)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations).
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

**When you hit ``STOP`` you will get a KeyInterrupt ``error``! No worries! :)**

---

## Step 6: Start evaluating:

This function assesses a trained model's performance for a specific shuffle or shuffles at a particular state or across all states on the dataset (images). The results are stored as a .csv file in a subdirectory under **evaluation-results**.

Look through the results in the **evaluation-results** directory. Additionally, review the very end of the output to identify train and test errors.

*How do you judge the model's performance?*

In [ ]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True, Shuffles=(shuffle,))

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler,
# so be sure your labels are good! (And you have trained enough ;)

## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos...
- Please see the repo and protocol instructions on how to refine your data!

---

## Step 7: Start Analyzing videos (Last one):
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides.

In [ ]:
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype=VideoType, shuffle=shuffle)

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
!pip install mpld3

In [ ]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType, shuffle=shuffle)

You can view these graphs in content folder, but they are also available for visualization within the notebook. Execute the following cell and attempt to interpret the plots.

*Is the performance of the model on our data satisfactory?*

Additionally, update the plot paths to visualize the graphs from the other videos.

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import mpld3
from IPython.core.display import display, HTML


# Define paths to the images
root = '/content/behavior&calcium-imaging-workshop/data/behavior-videos/plot-poses/day0'
plot_likelihood_path = f'{root}/plot-likelihood.png'
hist_path = f'{root}/hist.png'
plot_path = f'{root}/plot.png'
trajectory_path = f'{root}/trajectory.png'

# Create a figure and axis
fig, axes = plt.subplots(2, 2)

# Read and plot each image on the corresponding axis
images = [plot_likelihood_path, hist_path, plot_path, trajectory_path]
titles = ['Plot Likelihood', 'Histogram', 'Plot', 'Trajectory']

for ax, image, title in zip(axes.ravel(), images, titles):
    img = Image.open(image)
    ax.imshow(img)
    ax.set_title(title)
    ax.axis('off')

# Convert the Matplotlib figure to an interactive HTML plot using mpld3
html_plot = mpld3.fig_to_html(fig)

# Display the HTML plot
display(HTML(html_plot))

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype=VideoType, shhuffle=shuffle)

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Define the path to the labeled video
labelled_video_path = '/content/behavior&calcium-imaging-workshop/data/behavior-videos/day0croppedshortdownsampledDLC_resnet50_ss_vs_sscwJan30shuffle5_100100_p60_labeled.mp4'

# Read the video
video_capture = cv2.VideoCapture(labelled_video_path)

# Read the first frame
ret, frame = video_capture.read()

# Convert the frame from BGR to RGB (OpenCV reads images in BGR format)
frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# Create a figure and axis
fig, ax = plt.subplots()

# Display the frame using matplotlib
ax.imshow(frame_rgb)
ax.axis('off')  # Hide axes

# Convert the Matplotlib figure to an interactive HTML plot using mpld3
html_plot = mpld3.fig_to_html(fig)

# Display the HTML plot
display(HTML(html_plot))
